# Connect to Azure ML workspace

In [1]:
workspace="league-ws"
subscription_id="79451499-b2c0-4513-8dea-ef7f37173fbb"
resource_grp="league"

experiment_name = "league_predict_temp"
model_name = "leaguepredict.mml" # in case you want to change the name, keep the .mml extension
ws = Workspace(workspace_name = workspace,
               subscription_id = subscription_id,
               resource_group = resource_grp)

ws.get_details()

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: league-ws
Azure region: northcentralus
Subscription id: 79451499-b2c0-4513-8dea-ef7f37173fbb
Resource group: league

In [2]:
import azureml.core
from azureml.core import Workspace
from azureml.core.run import Run
from azureml.core.experiment import Experiment

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.74

In [3]:
import os
import numpy as np
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Read final matrix from cosmos db

In [4]:
## Dataset
# Champion Container
readConfig = {
"Endpoint" : "https://lolpredict.documents.azure.com:443/",
"Masterkey" : "lAb4WLi89cpXb7jGeMlLZtXUqvlGJip9NtKxDZMWHKUf0wasuAFknSgTxX3M7RAlK2uyFZjau7AJethdaBRDWw==",
"Database" : "league_extraction",
"Collection" : "FINAL_FEATURES_MATRIX_9112", 
}
final_features_matrix = spark.read.format("com.microsoft.azure.cosmosdb.spark").options(**readConfig).load()

In [5]:
dropped = ['_attachments',
 '_etag',
 '_rid',
 '_self',
 '_ts',
 'id',
 'label',
 'match_id',
 '100_CARRY_champion_inv',
 '100_CARRY_name',
 '100_CARRY_summonerId_inv',
 '100_JUNGLE_champion_inv',
 '100_JUNGLE_name',
 '100_JUNGLE_summonerId_inv',
 '100_MIDDLE_champion_inv',
 '100_MIDDLE_name',
 '100_MIDDLE_summonerId_inv',
 '100_SUPPORT_champion_inv',
 '100_SUPPORT_name',
 '100_SUPPORT_summonerId_inv',
 '100_TOP_champion_inv',
 '100_TOP_name',
 '100_TOP_summonerId_inv',
 '200_CARRY_champion_inv',
 '200_CARRY_name',
 '200_CARRY_summonerId_inv',
 '200_JUNGLE_champion_inv',
 '200_JUNGLE_name',
 '200_JUNGLE_summonerId_inv',
 '200_MIDDLE_champion_inv',
 '200_MIDDLE_name',
 '200_MIDDLE_summonerId_inv',
 '200_SUPPORT_champion_inv',
 '200_SUPPORT_name',
 '200_SUPPORT_summonerId_inv',
 '200_TOP_champion_inv',
 '200_TOP_name',
 '200_TOP_summonerId_inv']
columns = final_features_matrix.schema.names 
not_col = ['id','match_id','label','_attachments', '_etag', '_rid', '_self', '_ts']
features = [col for col in columns if col not in dropped]
features

Out[ 20 ]: ['100_CARRY_armorperlevel',
 '100_CARRY_attackdamageperlevel',
 '100_CARRY_attackrange',
 '100_CARRY_attackspeed',
 '100_CARRY_avg(assists)',
 '100_CARRY_avg(deaths)',
 '100_CARRY_avg(gold_earned)',
 '100_CARRY_avg(kills)',
 '100_CARRY_avg(totalDamageDealt)',
 '100_CARRY_avg(totalDamageTaken)',
 '100_CARRY_avg(totalMinionsKilled)',
 '100_CARRY_avg(totalTimeCrowdControlDealt)',
 '100_CARRY_avg(visionScore)',
 '100_CARRY_hpperlevel',
 '100_CARRY_spellblockperlevel',
 '100_JUNGLE_armorperlevel',
 '100_JUNGLE_attackdamageperlevel',
 '100_JUNGLE_attackrange',
 '100_JUNGLE_attackspeed',
 '100_JUNGLE_avg(assists)',
 '100_JUNGLE_avg(deaths)',
 '100_JUNGLE_avg(gold_earned)',
 '100_JUNGLE_avg(kills)',
 '100_JUNGLE_avg(totalDamageDealt)',
 '100_JUNGLE_avg(totalDamageTaken)',
 '100_JUNGLE_avg(totalMinionsKilled)',
 '100_JUNGLE_avg(totalTimeCrowdControlDealt)',
 '100_JUNGLE_avg(visionScore)',
 '100_JUNGLE_hpperlevel',
 '100_JUNGLE_spellblockperlevel',
 '100_MIDDLE_armorperlevel',
 '100_MIDDLE_attackdamageperlevel',
 '100_MIDDLE_attackrange',
 '100_MIDDLE_attackspeed',
 '100_MIDDLE_avg(assists)',
 '100_MIDDLE_avg(deaths)',
 '100_MIDDLE_avg(gold_earned)',
 '100_MIDDLE_avg(kills)',
 '100_MIDDLE_avg(totalDamageDealt)',
 '100_MIDDLE_avg(totalDamageTaken)',
 '100_MIDDLE_avg(totalMinionsKilled)',
 '100_MIDDLE_avg(totalTimeCrowdControlDealt)',
 '100_MIDDLE_avg(visionScore)',
 '100_MIDDLE_hpperlevel',
 '100_MIDDLE_spellblockperlevel',
 '100_SUPPORT_armorperlevel',
 '100_SUPPORT_attackdamageperlevel',
 '100_SUPPORT_attackrange',
 '100_SUPPORT_attackspeed',
 '100_SUPPORT_avg(assists)',
 '100_SUPPORT_avg(deaths)',
 '100_SUPPORT_avg(gold_earned)',
 '100_SUPPORT_avg(kills)',
 '100_SUPPORT_avg(totalDamageDealt)',
 '100_SUPPORT_avg(totalDamageTaken)',
 '100_SUPPORT_avg(totalMinionsKilled)',
 '100_SUPPORT_avg(totalTimeCrowdControlDealt)',
 '100_SUPPORT_avg(visionScore)',
 '100_SUPPORT_hpperlevel',
 '100_SUPPORT_spellblockperlevel',
 '100_TOP_armorperlevel',
 '100_TOP_attackdamageperlevel',
 '100_TOP_attackrange',
 '100_TOP_attackspeed',
 '100_TOP_avg(assists)',
 '100_TOP_avg(deaths)',
 '100_TOP_avg(gold_earned)',
 '100_TOP_avg(kills)',
 '100_TOP_avg(totalDamageDealt)',
 '100_TOP_avg(totalDamageTaken)',
 '100_TOP_avg(totalMinionsKilled)',
 '100_TOP_avg(totalTimeCrowdControlDealt)',
 '100_TOP_avg(visionScore)',
 '100_TOP_hpperlevel',
 '100_TOP_spellblockperlevel',
 '200_CARRY_armorperlevel',
 '200_CARRY_attackdamageperlevel',
 '200_CARRY_attackrange',
 '200_CARRY_attackspeed',
 '200_CARRY_avg(assists)',
 '200_CARRY_avg(deaths)',
 '200_CARRY_avg(gold_earned)',
 '200_CARRY_avg(kills)',
 '200_CARRY_avg(totalDamageDealt)',
 '200_CARRY_avg(totalDamageTaken)',
 '200_CARRY_avg(totalMinionsKilled)',
 '200_CARRY_avg(totalTimeCrowdControlDealt)',
 '200_CARRY_avg(visionScore)',
 '200_CARRY_hpperlevel',
 '200_CARRY_spellblockperlevel',
 '200_JUNGLE_armorperlevel',
 '200_JUNGLE_attackdamageperlevel',
 '200_JUNGLE_attackrange',
 '200_JUNGLE_attackspeed',
 '200_JUNGLE_avg(assists)',
 '200_JUNGLE_avg(deaths)',
 '200_JUNGLE_avg(gold_earned)',
 '200_JUNGLE_avg(kills)',
 '200_JUNGLE_avg(totalDamageDealt)',
 '200_JUNGLE_avg(totalDamageTaken)',
 '200_JUNGLE_avg(totalMinionsKilled)',
 '200_JUNGLE_avg(totalTimeCrowdControlDealt)',
 '200_JUNGLE_avg(visionScore)',
 '200_JUNGLE_hpperlevel',
 '200_JUNGLE_spellblockperlevel',
 '200_MIDDLE_armorperlevel',
 '200_MIDDLE_attackdamageperlevel',
 '200_MIDDLE_attackrange',
 '200_MIDDLE_attackspeed',
 '200_MIDDLE_avg(assists)',
 '200_MIDDLE_avg(deaths)',
 '200_MIDDLE_avg(gold_earned)',
 '200_MIDDLE_avg(kills)',
 '200_MIDDLE_avg(totalDamageDealt)',
 '200_MIDDLE_avg(totalDamageTaken)',
 '200_MIDDLE_avg(totalMinionsKilled)',
 '200_MIDDLE_avg(totalTimeCrowdControlDealt)',
 '200_MIDDLE_avg(visionScore)',
 '200_MIDDLE_hpperlevel',
 '200_MIDDLE_spellblockperlevel',
 '200_SUPPORT_armorperlevel',
 '200_SUPPORT_attackdamageperlevel',
 '200_SUPPORT_attackrange',
 '200_SUPPORT_attackspeed',
 '200_SUPPORT_avg(assists)',
 '200_SUPPORT_avg(deaths)

## Machine learning model

In [6]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = features,outputCol='features')
class_df = vectorAssembler.transform(final_features_matrix)
class_df = class_df.select(['features','label'])
(train_df, test_df) = class_df.randomSplit([0.7,0.3],seed=42)

In [7]:
train_df.count()

Out[ 22 ]: 6416

In [8]:
test_df.count()

Out[ 23 ]: 2696

In [9]:
#Train
lr = LogisticRegression()
model = lr.fit(train_df)
# Print the coefficients and intercept for logistic regression
#   print("Coefficients: " + str(model.coefficients))
#   print("Intercept: " + str(model.intercept))
#Train results
train_sum = model.summary
accuracy = train_sum.accuracy
print("Train Accuracy {}".format(accuracy))
train_sum.roc.show()
print("Train areaUnderROC: " + str(train_sum.areaUnderROC))

#test prediction 
predictions = model.transform(test_df)
predictions.select("prediction","label")

#Test evaluation
from pyspark.ml.evaluation import BinaryClassificationEvaluator
# evaluate. note only 2 metrics are supported out of the box by Spark ML.
bce = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction')
au_roc = bce.setMetricName('areaUnderROC').evaluate(predictions)
au_prc = bce.setMetricName('areaUnderPR').evaluate(predictions)

truePositive = predictions.select("label").filter("label = 1 and prediction = 1").count()
falsePositive = predictions.select("label").filter("label = 0 and prediction = 1").count()
trueNegative = predictions.select("label").filter("label = 0 and prediction = 0").count()
falseNegative = predictions.select("label").filter("label = 1 and prediction = 0").count()

precision = truePositive/(truePositive + falsePositive)
recall = truePositive/(truePositive + falseNegative)

print("truePositive: " + str(truePositive))
print("falsePositive: " + str(falsePositive))
print("trueNegative: " + str(trueNegative))
print("falseNegative: " + str(falseNegative))
print("precision: " + str(precision))
print("recall: " + str(recall))

# "Official" statical measurement (the closer to 1, the better)
print("Area under ROC: {}".format(au_roc))
print("Area Under PR: {}".format(au_prc))


Train Accuracy 0.8552057356608479
+--------------------+--------------------+
 FPR| TPR|
+--------------------+--------------------+
 0.0| 0.0|
 0.0|0.019931485518530054|
 0.0| 0.03986297103706011|
 0.0| 0.05979445655559016|
 0.0| 0.07972594207412022|
 0.0| 0.09965742759265027|
 0.0| 0.11958891311118032|
 0.0| 0.13952039862971036|
3.120124804992199...| 0.1591404546870134|
3.120124804992199...| 0.17907194020554346|
3.120124804992199...| 0.1990034257240735|
3.120124804992199...| 0.21893491124260356|
3.120124804992199...| 0.2388663967611336|
3.120124804992199...| 0.25879788227966366|
6.240249609984399E-4| 0.27841793833696665|
6.240249609984399E-4| 0.29834942385549673|
 9.3603744149766E-4| 0.3179694799127997|
0.001248049921996...| 0.33758953597010277|
 0.00187207488299532| 0.35689816256617873|
 0.00218408736349454| 0.3765182186234818|
+--------------------+--------------------+
only showing top 20 rows

Train areaUnderROC: 0.9396859275180725
truePositive: 1181
falsePositive: 209
trueNegative: 1101
falseNegative: 205
precision: 0.8496402877697842
recall: 0.8520923520923521
Area under ROC: 0.9311589174184552
Area Under PR: 0.9366853150983515